In [1]:
import os
import numpy as np
from dataclasses import dataclass
from typing import Callable, Tuple, Optional, List, Dict

os.environ["OMP_NUM_THREADS"] = "1"  # чтобы убрать варнинг MKL

from sklearn.cluster import KMeans
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings("ignore", category=ConvergenceWarning)


# ==========================
#   БАЗОВЫЕ СТРУКТУРЫ ДАННЫХ
# ==========================

@dataclass
class VariableBounds:
    """Границы вещественных переменных."""
    low: np.ndarray   # shape (n_real,)
    high: np.ndarray  # shape (n_real,)


@dataclass
class DiscreteSpec:
    """Описание дискретных переменных: по каждой - список допустимых значений."""
    values: List[np.ndarray]  # len=n_disc, каждый np.array возможных значений


@dataclass
class GAConfig:
    pop_size: int
    n_generations: int
    crossover_rate: float = 0.8
    mutation_rate: float = 0.1
    tournament_size: int = 3

    # кластеризация
    n_clusters: int = 5

    # фильтрация кластеров
    alpha_N: float = 1.0   # порог по размеру кластера: N_k < alpha_N * N_mean
    alpha_F: float = 1.0   # порог по пригодности: F_k > alpha_F * F_mean

    # ICE-порог
    theta_ice: float = 1.0

    # бинарное кодирование
    desired_step: float = 0.01  # желаемый шаг по умолчанию

    # минимальный размер популяции (для иммиграции)
    min_pop_size: int = 20

    # размер элитного архива
    elite_archive_size: int = 10


# ==========================
#   АДАПТИВНЫЙ ГЕНЕТИЧЕСКИЙ АЛГОРИТМ
# ==========================

class AdaptiveGeneticAlgorithm:
    """
    Адаптивный ГА с динамической коррекцией пространства поиска (AGA-DSP).
    Реализует:
      - бинарное кодирование вещественной части с шагом (b-a)/(2^N-1),
      - кластеризацию по вещественной части,
      - исключение неэффективных кластеров,
      - критерий возможности коррекции пространства,
      - коррекцию границ по ICE,
      - обновление схемы кодирования и перекодирование особей,
      - иммиграцию и элитный архив.
    """

    def __init__(
        self,
        fitness_fn: Callable[[np.ndarray, np.ndarray], float],
        real_bounds: VariableBounds,
        disc_spec: Optional[DiscreteSpec],
        config: GAConfig,
        n_bits_per_var: Optional[np.ndarray] = None,
        random_state: Optional[int] = None,
    ):
        """
        fitness_fn(x_real, x_disc) -> float
          x_real : (n_real,) вещественная часть
          x_disc : (n_disc,) дискретная часть (по индексам в DiscreteSpec)
        """
        self.fitness_fn = fitness_fn
        self.real_bounds = real_bounds
        self.disc_spec = disc_spec
        self.config = config
        self.rng = np.random.default_rng(random_state)

        self.n_real = len(real_bounds.low)
        self.n_disc = len(disc_spec.values) if disc_spec is not None else 0

        # число бит на каждую вещественную переменную
        if n_bits_per_var is None:
            self.n_bits_per_var = self._init_bits(real_bounds, config.desired_step)
        else:
            self.n_bits_per_var = np.asarray(n_bits_per_var, dtype=int)

        # текущие глобальные границы для вещественных переменных
        self.current_low = real_bounds.low.copy()
        self.current_high = real_bounds.high.copy()

        # текущие шаги дискретизации
        self.current_step = self._recompute_step(self.current_low, self.current_high)

        # длина бинарного генотипа для вещественной части
        self.real_genome_length = int(np.sum(self.n_bits_per_var))

        # элитный архив (лучшие особи за всю историю)
        self.elite_bits: Optional[np.ndarray] = None
        self.elite_disc: Optional[np.ndarray] = None
        self.elite_fit: Optional[np.ndarray] = None

    # -------------------------------------------------------------------------
    # Инициализация бинарного кодирования
    # -------------------------------------------------------------------------

    def _init_bits(self, bounds: VariableBounds, desired_step: float) -> np.ndarray:
        low, high = bounds.low, bounds.high
        n_real = len(low)
        n_bits = np.zeros(n_real, dtype=int)
        for i in range(n_real):
            interval = high[i] - low[i]
            raw = np.log2(max(interval / desired_step, 2.0))
            n_bits[i] = int(np.ceil(raw))
        return n_bits

    def _recompute_step(self, low: np.ndarray, high: np.ndarray) -> np.ndarray:
        """stepC_i = (high_i - low_i) / (2^N_i - 1)."""
        step = np.empty_like(low, dtype=float)
        for i in range(len(low)):
            interval = high[i] - low[i]
            step[i] = interval / (2 ** self.n_bits_per_var[i] - 1)
        return step

    # -------------------------------------------------------------------------
    # Генотип/фенотип: бинарное кодирование вещественной части
    # -------------------------------------------------------------------------

    def _encode_real(self, x_real: np.ndarray) -> np.ndarray:
        """
        Кодирование вещественного вектора в бинарную строку.
        x_real shape: (n_real,)
        return shape: (real_genome_length,), dtype=bool
        """
        bits = []
        for i in range(self.n_real):
            xi = x_real[i]
            # ограничиваем в текущих границах
            xi = np.clip(xi, self.current_low[i], self.current_high[i])
            t = round((xi - self.current_low[i]) / self.current_step[i])
            t = int(np.clip(t, 0, 2 ** self.n_bits_per_var[i] - 1))
            # перевод в двоичный код фиксированной длины
            b = np.array(list(np.binary_repr(t, width=self.n_bits_per_var[i])), dtype='U1') == '1'
            bits.append(b)
        return np.concatenate(bits)

    def _decode_real(self, bits: np.ndarray) -> np.ndarray:
        """
        Декодирование бинарной строки в вещественный вектор.
        bits shape: (real_genome_length,), dtype=bool
        """
        x_real = np.empty(self.n_real, dtype=float)
        idx = 0
        for i in range(self.n_real):
            nb = self.n_bits_per_var[i]
            segment = bits[idx:idx+nb]
            idx += nb
            # двоичный -> целое
            t = 0
            for bit in segment:
                t = (t << 1) | int(bit)
            x_real[i] = self.current_low[i] + t * self.current_step[i]
        return x_real

    def _init_population(self) -> Tuple[np.ndarray, np.ndarray]:
        """Инициализация популяции: бинарная вещественная часть + дискретная часть."""
        pop_real_bits = np.zeros((self.config.pop_size, self.real_genome_length), dtype=bool)
        if self.n_disc > 0:
            pop_disc_idx = np.zeros((self.config.pop_size, self.n_disc), dtype=int)
        else:
            pop_disc_idx = np.zeros((self.config.pop_size, 0), dtype=int)

        for m in range(self.config.pop_size):
            # вещественная часть
            x_real = self.rng.uniform(self.real_bounds.low, self.real_bounds.high)
            pop_real_bits[m] = self._encode_real(x_real)
            # дискретная часть: случайные индексы
            if self.n_disc > 0:
                for j, vals in enumerate(self.disc_spec.values):
                    pop_disc_idx[m, j] = self.rng.integers(0, len(vals))
        return pop_real_bits, pop_disc_idx

    def _decode_population(self, pop_real_bits, pop_disc_idx):
        """Декодировать всю популяцию в фенотипы (x_real, x_disc_values)."""
        X_real = np.zeros((pop_real_bits.shape[0], self.n_real), dtype=float)
        if self.n_disc > 0:
            X_disc = np.zeros((pop_real_bits.shape[0], self.n_disc), dtype=object)
        else:
            X_disc = np.zeros((pop_real_bits.shape[0], 0), dtype=object)

        for m in range(pop_real_bits.shape[0]):
            X_real[m] = self._decode_real(pop_real_bits[m])
            if self.n_disc > 0:
                for j, vals in enumerate(self.disc_spec.values):
                    X_disc[m, j] = vals[pop_disc_idx[m, j]]
        return X_real, X_disc

    # -------------------------------------------------------------------------
    # GA операторы: оценка, селекция, кроссовер, мутация
    # -------------------------------------------------------------------------

    def _evaluate_fitness(self, pop_real_bits, pop_disc_idx) -> np.ndarray:
        X_real, X_disc = self._decode_population(pop_real_bits, pop_disc_idx)
        fitness = np.zeros(X_real.shape[0], dtype=float)
        for m in range(X_real.shape[0]):
            if self.n_disc > 0:
                fitness[m] = self.fitness_fn(X_real[m], X_disc[m])
            else:
                fitness[m] = self.fitness_fn(X_real[m], np.array([]))
        return fitness

    def _tournament_selection(self, fitness: np.ndarray) -> np.ndarray:
        """Возвращает индексы выбранных родителей (турнир, безопасный при маленькой популяции)."""
        idx = np.arange(len(fitness))
        n_pop = len(idx)
        if n_pop == 0:
            raise ValueError("Пустая популяция в _tournament_selection")

        # не больше, чем размер популяции
        t_size = min(self.config.tournament_size, n_pop)

        selected = np.empty(self.config.pop_size, dtype=int)
        for i in range(self.config.pop_size):
            competitors = self.rng.choice(idx, size=t_size, replace=False)
            best = competitors[np.argmin(fitness[competitors])]  # minimization
            selected[i] = best
        return selected

    def _one_point_crossover(self, parent1_bits, parent2_bits):
        if self.rng.random() > self.config.crossover_rate:
            return parent1_bits.copy(), parent2_bits.copy()
        point = self.rng.integers(1, len(parent1_bits))
        child1 = np.concatenate([parent1_bits[:point], parent2_bits[point:]])
        child2 = np.concatenate([parent2_bits[:point], parent1_bits[point:]])
        return child1, child2

    def _mutation_bits(self, bits):
        for i in range(len(bits)):
            if self.rng.random() < self.config.mutation_rate:
                bits[i] = ~bits[i]
        return bits

    def _mutation_disc(self, disc_idx):
        if self.n_disc == 0:
            return disc_idx
        for j, vals in enumerate(self.disc_spec.values):
            if self.rng.random() < self.config.mutation_rate:
                disc_idx[j] = self.rng.integers(0, len(vals))
        return disc_idx

    def _reproduce(self, pop_real_bits, pop_disc_idx, fitness):
        selected_idx = self._tournament_selection(fitness)
        new_real_bits = np.zeros_like(pop_real_bits)
        new_disc_idx = np.zeros_like(pop_disc_idx)

        # попарный кроссовер
        for i in range(0, self.config.pop_size, 2):
            p1 = selected_idx[i]
            if i + 1 < self.config.pop_size:
                p2 = selected_idx[i+1]
            else:
                p2 = selected_idx[0]
            c1_bits, c2_bits = self._one_point_crossover(pop_real_bits[p1], pop_real_bits[p2])
            c1_bits = self._mutation_bits(c1_bits)
            c2_bits = self._mutation_bits(c2_bits)

            c1_disc = self._mutation_disc(pop_disc_idx[p1].copy())
            c2_disc = self._mutation_disc(pop_disc_idx[p2].copy())

            new_real_bits[i] = c1_bits
            new_disc_idx[i] = c1_disc
            if i + 1 < self.config.pop_size:
                new_real_bits[i+1] = c2_bits
                new_disc_idx[i+1] = c2_disc

        return new_real_bits, new_disc_idx

    # -------------------------------------------------------------------------
    # Кластеризация
    # -------------------------------------------------------------------------

    def _clusterize(self, X_real: np.ndarray):
        """
        Возвращает метки кластеров и список описаний кластеров.
        Корректно обрабатывает случаи, когда особей меньше, чем n_clusters.
        """
        n_samples = X_real.shape[0]
        if n_samples == 0:
            return np.array([], dtype=int), []

        # число кластеров не может быть больше числа точек
        n_clusters = min(self.config.n_clusters, n_samples)

        # если кластер всего один – просто один диапазон без KMeans
        if n_clusters == 1:
            labels = np.zeros(n_samples, dtype=int)
            bounds_low = X_real.min(axis=0)
            bounds_high = X_real.max(axis=0)
            clusters_info = [{
                "idx": np.arange(n_samples),
                "bounds_low": bounds_low,
                "bounds_high": bounds_high
            }]
            return labels, clusters_info

        # обычный случай: 2..n_samples кластеров
        kmeans = KMeans(
            n_clusters=n_clusters,
            n_init=10,
            random_state=int(self.rng.integers(0, 1_000_000_000))
        )
        labels = kmeans.fit_predict(X_real)

        clusters_info = []
        for k in range(n_clusters):
            idx = np.where(labels == k)[0]
            if len(idx) == 0:
                clusters_info.append({
                    "idx": idx,
                    "bounds_low": None,
                    "bounds_high": None
                })
                continue

            cluster_points = X_real[idx]
            bounds_low = cluster_points.min(axis=0)
            bounds_high = cluster_points.max(axis=0)
            clusters_info.append({
                "idx": idx,
                "bounds_low": bounds_low,
                "bounds_high": bounds_high
            })

        return labels, clusters_info

    # -------------------------------------------------------------------------
    # Элитный архив
    # -------------------------------------------------------------------------

    def _update_elite_archive(
        self,
        pop_real_bits: np.ndarray,
        pop_disc_idx: np.ndarray,
        fitness: np.ndarray
    ):
        """Обновляет архив лучших особей (по всей истории)."""
        if pop_real_bits.shape[0] == 0:
            return

        # индексы особей текущего поколения, отсортированные по fitness
        order = np.argsort(fitness)  # минимум лучше
        k = min(self.config.elite_archive_size, len(order))
        best_idx = order[:k]

        cand_bits = pop_real_bits[best_idx].copy()
        cand_disc = pop_disc_idx[best_idx].copy()
        cand_fit = fitness[best_idx].copy()

        if self.elite_bits is None:
            # первый раз инициализируем архив
            self.elite_bits = cand_bits
            self.elite_disc = cand_disc
            self.elite_fit = cand_fit
        else:
            # склеиваем старые + новые
            all_bits = np.vstack([self.elite_bits, cand_bits])
            all_disc = np.vstack([self.elite_disc, cand_disc])
            all_fit = np.concatenate([self.elite_fit, cand_fit])

            # снова отсортировать и оставить top-K
            order_all = np.argsort(all_fit)
            k_all = min(self.config.elite_archive_size, len(order_all))
            keep = order_all[:k_all]

            self.elite_bits = all_bits[keep]
            self.elite_disc = all_disc[keep]
            self.elite_fit = all_fit[keep]

    # -------------------------------------------------------------------------
    # Иммиграция (поддержание минимального размера популяции)
    # -------------------------------------------------------------------------

    def _ensure_min_population(
        self,
        pop_real_bits: np.ndarray,
        pop_disc_idx: np.ndarray,
        worse_than_prev: bool
    ):
        """
        Если популяция стала меньше config.min_pop_size,
        досыпаем особей:
          - если поколение ухудшилось и есть элита — берем из архива,
          - остаток добиваем случайными "иммигрантами".
        """
        n_current = pop_real_bits.shape[0]
        target = max(self.config.min_pop_size, n_current)

        if n_current >= target:
            return pop_real_bits, pop_disc_idx

        n_add = target - n_current

        new_real_chunks = []
        new_disc_chunks = []

        # 1) если стало хуже и есть элитный архив — добавляем лучших
        if worse_than_prev and (self.elite_bits is not None) and (self.elite_bits.shape[0] > 0):
            n_from_elite = min(n_add, self.elite_bits.shape[0])
            elite_idx = np.arange(self.elite_bits.shape[0])
            chosen = self.rng.choice(elite_idx, size=n_from_elite, replace=False)

            elite_bits = self.elite_bits[chosen].copy()
            if self.n_disc > 0:
                elite_disc = self.elite_disc[chosen].copy()
            else:
                elite_disc = np.zeros((n_from_elite, 0), dtype=int)

            new_real_chunks.append(elite_bits)
            new_disc_chunks.append(elite_disc)
            n_add -= n_from_elite

        # 2) оставшиеся слоты добиваем случайными особями
        if n_add > 0:
            rand_real_bits = np.zeros((n_add, self.real_genome_length), dtype=bool)
            if self.n_disc > 0:
                rand_disc_idx = np.zeros((n_add, self.n_disc), dtype=int)
            else:
                rand_disc_idx = np.zeros((n_add, 0), dtype=int)

            for m in range(n_add):
                x_real = self.rng.uniform(self.current_low, self.current_high)
                rand_real_bits[m] = self._encode_real(x_real)
                if self.n_disc > 0:
                    for j, vals in enumerate(self.disc_spec.values):
                        rand_disc_idx[m, j] = self.rng.integers(0, len(vals))

            new_real_chunks.append(rand_real_bits)
            new_disc_chunks.append(rand_disc_idx)

        if new_real_chunks:
            new_real_bits = np.vstack(new_real_chunks)
            new_disc_idx = np.vstack(new_disc_chunks)
            pop_real_bits = np.vstack([pop_real_bits, new_real_bits])
            pop_disc_idx = np.vstack([pop_disc_idx, new_disc_idx])

        return pop_real_bits, pop_disc_idx

    # -------------------------------------------------------------------------
    # Основной run()
    # -------------------------------------------------------------------------

    def run(self):
        """
        Запуск AGA-DSP на одну задачу.
        Возвращает:
          best_solution  -- (x_real, x_disc или None)
          best_fitness   -- лучшее значение целевой функции
          history        -- список кортежей:
                            (generation, best_fitness, pop_before, pop_after, n_clusters)
        """
        # каждый запуск начинаем с исходных границ
        self.current_low = self.real_bounds.low.copy()
        self.current_high = self.real_bounds.high.copy()
        self.current_step = self._recompute_step(self.current_low, self.current_high)

        # сохранить исходный размер популяции, потому что внутри run он меняется
        initial_pop_size = self.config.pop_size

        pop_real_bits, pop_disc_idx = self._init_population()

        best_fitness = np.inf
        best_solution = None
        history = []

        # для критерия коррекции: предыдущие глобальные границы
        prev_low = self.current_low.copy()
        prev_high = self.current_high.copy()

        prev_gen_best = np.inf  # лучший фитнес на предыдущем поколении

        for g in range(1, self.config.n_generations + 1):
            # === 1. Оценка текущей популяции ===
            fitness = self._evaluate_fitness(pop_real_bits, pop_disc_idx)
            pop_before = len(fitness)

            # лучший на поколении
            idx_best = np.argmin(fitness)
            gen_best = fitness[idx_best]

            # глобально лучший
            if gen_best < best_fitness:
                best_fitness = gen_best
                br, bd = self._decode_population(
                    pop_real_bits[idx_best:idx_best + 1],
                    pop_disc_idx[idx_best:idx_best + 1]
                )
                best_solution = (br[0], bd[0] if self.n_disc > 0 else None)

            # сравниваем с прошлым поколением: стало ли хуже?
            worse_than_prev = (g > 1) and (gen_best > prev_gen_best)
            prev_gen_best = gen_best

            # обновляем элитный архив текущими лучшими
            self._update_elite_archive(pop_real_bits, pop_disc_idx, fitness)

            # === 2. Генетические операторы (получаем детей) ===
            pop_real_bits, pop_disc_idx = self._reproduce(pop_real_bits, pop_disc_idx, fitness)

            # === 3. Декодируем для кластеризации ===
            X_real, _ = self._decode_population(pop_real_bits, pop_disc_idx)

            # === 4. Кластеризация потомков ===
            _, clusters_info = self._clusterize(X_real)

            # === 5. Фильтрация неэффективных кластеров ===
            pop_real_bits, pop_disc_idx, X_real_filtered, fitness_filtered, clusters_info_filtered = \
                self._filter_clusters(pop_real_bits, pop_disc_idx, X_real, fitness, clusters_info)

            # если популяция опустела — выход
            if pop_real_bits.shape[0] == 0:
                print(f"Gen {g:4d} | POPULATION EMPTY after filtering, stop.")
                # для истории всё равно сохраним состояние (pop_after=0, K=0)
                history.append((g, gen_best, pop_before, 0, 0))
                break

            # обновляем размер популяции (после фильтрации)
            self.config.pop_size = pop_real_bits.shape[0]

            # === 6. Критерий возможности коррекции пространства ===
            do_correct = self._should_correct(prev_low, prev_high, clusters_info_filtered)

            if do_correct:
                # корректировка границ по ICE
                self._correct_bounds_ice(X_real_filtered, fitness_filtered, clusters_info_filtered)

                # обновление шагов и перекодирование популяции
                self.current_step = self._recompute_step(self.current_low, self.current_high)
                pop_real_bits = self._reencode_population(X_real_filtered)

                # обновляем prev_* на новую итерацию
                prev_low = self.current_low.copy()
                prev_high = self.current_high.copy()

            # === 7. Иммиграция / поддержание минимального размера популяции ===
            pop_real_bits, pop_disc_idx = self._ensure_min_population(
                pop_real_bits,
                pop_disc_idx,
                worse_than_prev=worse_than_prev
            )
            # обновить pop_size после иммиграции
            self.config.pop_size = pop_real_bits.shape[0]

            pop_after = self.config.pop_size
            k_after = len(clusters_info_filtered)

            # === 8. Сохраняем в историю и выводим в консоль ===
            history.append((g, gen_best, pop_before, pop_after, k_after))

            print(
                f"Gen {g:4d} | best = {gen_best:.6e} | "
                f"pop_before = {pop_before:4d} | pop_after = {pop_after:4d} | "
                f"K = {k_after}"
            )

        # восстановить исходный pop_size для следующего запуска
        self.config.pop_size = initial_pop_size

        return best_solution, best_fitness, history


    # -------------------------------------------------------------------------
    # Фильтрация кластеров по размеру и пригодности
    # -------------------------------------------------------------------------

    def _filter_clusters(self, pop_real_bits, pop_disc_idx, X_real, fitness, clusters_info):
        N_k = np.array([len(c["idx"]) for c in clusters_info], dtype=int)
        # выкидываем полностью пустые
        non_empty_mask = N_k > 0

        N_mean = np.mean(N_k[non_empty_mask]) if np.any(non_empty_mask) else 0.0
        F_mean = np.mean(fitness) if len(fitness) > 0 else 0.0

        keep_indices = np.ones(len(fitness), dtype=bool)
        new_clusters_info = []

        for c in clusters_info:
            idx = c["idx"]
            if len(idx) == 0:
                continue
            N_c = len(idx)
            F_c = np.mean(fitness[idx])
            if (N_c < self.config.alpha_N * N_mean) and (F_c > self.config.alpha_F * F_mean):
                # NOTE: задача минимизации, "хуже" — больше F
                keep_indices[idx] = False
            else:
                new_clusters_info.append(c)

        pop_real_bits_new = pop_real_bits[keep_indices]
        pop_disc_idx_new = pop_disc_idx[keep_indices]
        X_real_new = X_real[keep_indices]
        fitness_new = fitness[keep_indices]

        # обновляем idx в кластерах относительно новой популяции
        mapping = np.where(keep_indices)[0]
        for c in new_clusters_info:
            c["idx"] = np.array(
                [np.where(mapping == i)[0][0] for i in c["idx"] if keep_indices[i]],
                dtype=int
            )

        return pop_real_bits_new, pop_disc_idx_new, X_real_new, fitness_new, new_clusters_info

    # -------------------------------------------------------------------------
    # Критерий возможности коррекции пространства
    # -------------------------------------------------------------------------

    def _should_correct(self, prev_low, prev_high, clusters_info) -> bool:
        if len(clusters_info) == 0:
            return False
        used_low = np.full(self.n_real, np.inf)
        used_high = np.full(self.n_real, -np.inf)
        any_used = False

        for c in clusters_info:
            if c["bounds_low"] is None:
                continue
            used_low = np.minimum(used_low, c["bounds_low"])
            used_high = np.maximum(used_high, c["bounds_high"])
            any_used = True

        if not any_used:
            return False

        # длина занятых отрезков (по проекции)
        L_used = np.maximum(used_high - used_low, 0.0)
        # свободная часть относительно prev
        L_total = prev_high - prev_low
        L_free = np.maximum(L_total - L_used, 0.0)

        return np.sum(L_free) > np.sum(L_used)

    # -------------------------------------------------------------------------
    # Коррекция границ по ICE
    # -------------------------------------------------------------------------

    def _correct_bounds_ice(self, X_real, fitness, clusters_info):
        # глобальные границы пересчитаем после прохода по кластерам
        new_low = self.current_low.copy()
        new_high = self.current_high.copy()

        F_total = np.sum(fitness)
        if F_total <= 0:
            # защита от деления на ноль (в случае странной функции)
            return

        for c in clusters_info:
            idx = c["idx"]
            if len(idx) == 0:
                continue
            cluster_points = X_real[idx]
            F_cluster = fitness[idx]

            # относительные пригодности
            RE = F_cluster / F_total
            RE_cluster_sum = np.sum(RE)
            if RE_cluster_sum <= 0:
                continue
            ICE = RE / RE_cluster_sum

            threshold = self.config.theta_ice / len(idx)
            effective_mask = ICE >= threshold
            if not np.any(effective_mask):
                # если никого не осталось, пропускаем
                continue

            effective_points = cluster_points[effective_mask]
            effective_ice = ICE[effective_mask]

            # начинаем с текущих границ кластера (по исходным вычисленным)
            cl_low = np.min(cluster_points, axis=0)
            cl_high = np.max(cluster_points, axis=0)

            # по каждой координате корректируем
            for i in range(self.n_real):
                coord = cluster_points[:, i]
                ice_coord = ICE

                # левый
                idx_min = np.argmin(coord)
                L_val = coord[idx_min]
                L_ice = ice_coord[idx_min]

                # если неэффективен -> заменить на лучший из эффективных
                if L_ice < threshold:
                    eff_coord = effective_points[:, i]
                    best_eff_idx = np.argmin(eff_coord)
                    L_val = eff_coord[best_eff_idx]
                    L_ice = effective_ice[best_eff_idx]

                # правый
                idx_max = np.argmax(coord)
                H_val = coord[idx_max]
                H_ice = ice_coord[idx_max]

                if H_ice < threshold:
                    eff_coord = effective_points[:, i]
                    best_eff_idx = np.argmax(eff_coord)
                    H_val = eff_coord[best_eff_idx]
                    H_ice = effective_ice[best_eff_idx]

                rng = max(H_val - L_val, 0.0)
                delta_l = -rng * L_ice
                delta_h = +rng * H_ice

                cl_low[i] = L_val + delta_l
                cl_high[i] = H_val + delta_h

            # сливаем с глобальными границами
            new_low = np.minimum(new_low, cl_low)
            new_high = np.maximum(new_high, cl_high)

        self.current_low = new_low
        self.current_high = new_high

    # -------------------------------------------------------------------------
    # Перекодирование после изменения границ
    # -------------------------------------------------------------------------

    def _reencode_population(self, X_real: np.ndarray) -> np.ndarray:
        pop_bits = np.zeros((X_real.shape[0], self.real_genome_length), dtype=bool)
        for m in range(X_real.shape[0]):
            pop_bits[m] = self._encode_real(X_real[m])
        return pop_bits




In [4]:
import sys, os

# путь к папке, где лежит папка cec2017 и ноутбук
sys.path.append(os.getcwd())

from cec2017 import basic, transforms, hybrid

print(hybrid.__file__)  # для проверки, откуда он грузится

import numpy as np

import warnings

# чтобы sklearn KMeans не орал про MKL
os.environ["OMP_NUM_THREADS"] = "1"

# глушим основные типы предупреждений
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)


def _shuffle_and_partition(x, shuffle, partitions):
    """
    First applies the given permutation, then splits x into partitions given
    the percentages.

    Args:
        x (array): Input vector.
        shuffle (array): Shuffle vector.
        partitions (list): List of percentages. Assumed to add up to 1.0.

    Returns:
        (list of arrays): The partitions of x after shuffling.
    """
    nx = len(x)
    # shuffle
    xs = np.zeros(x.shape)
    for i in range(0, nx):
        xs[i] = x[shuffle[i]]
    # and partition
    parts = []
    start, end = 0, 0
    for p in partitions[:-1]:
        end = start + int(np.ceil(p * nx))
        parts.append(xs[start:end])
        start = end
    parts.append(xs[end:])
    return parts



def f1(x, rotation=None, shift=None):
    """
    Shifted and Rotated Bent Cigar Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][0]
    if shift is None:
        shift = transforms.shifts[0][:nx]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.bent_cigar(x_transformed) + 100.0


def f2(x, rotation=None, shift=None):
    """
    (Deprecated) Shifted and Rotated Sum of Different Power Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    if 'warned' not in f2.__dict__:
        f2.warned = True
        print('WARNING: f2 has been deprecated from the CEC 2017 benchmark suite')

    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][1]
    if shift is None:
        shift = transforms.shifts[1][:nx]
    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.sum_diff_pow(x_transformed) + 200.0


def f3(x, rotation=None, shift=None):
    """
    Shifted and Rotated Zakharov Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][2]
    if shift is None:
        shift = transforms.shifts[2][:nx]
    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.zakharov(x_transformed) + 300.0


def f4(x, rotation=None, shift=None):
    """
    Shifted and Rotated Rosenbrock's Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][3]
    if shift is None:
        shift = transforms.shifts[3][:nx]
    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.rosenbrock(x_transformed) + 400.0


def f5(x, rotation=None, shift=None):
    """
    Shifted and Rotated Rastrigin's Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][4]
    if shift is None:
        shift = transforms.shifts[4][:nx]
    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.rastrigin(x_transformed) + 500.0


def f6(x, rotation=None, shift=None):
    """
    Shifted and Rotated Schaffer's F7 Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][5]
    if shift is None:
        shift = transforms.shifts[5][:nx]
    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.schaffers_f7(x_transformed) + 600.0


def f7(x, rotation=None, shift=None):
    """
    Shifted and Rotated Lunacek Bi-Rastrigin's Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][6]
    if shift is None:
        shift = transforms.shifts[6][:nx]
    # pass the shift and rotation directly to the function
    return basic.lunacek_bi_rastrigin(x, shift, rotation) + 700.0


def f8(x, rotation=None, shift=None):
    """
    Shifted and Rotated Non-Continuous Rastrigin’s Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][7]
    if shift is None:
        shift = transforms.shifts[7][:nx]
    # pass the shift and rotation directly to the function
    return basic.non_cont_rastrigin(x, shift, rotation) + 800.0


def f9(x, rotation=None, shift=None):
    """
    Shifted and Rotated Levy Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][8]
    if shift is None:
        shift = transforms.shifts[8][:nx]
    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.levy(x_transformed) + 900.0


def f10(x, rotation=None, shift=None):
    """
    Shifted and Rotated Schwefel’s Function

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][9]
    if shift is None:
        shift = transforms.shifts[9][:nx]
    x_transformed = transforms.shift_rotate(x, shift, rotation)
    return basic.modified_schwefel(x_transformed) + 1000.0



def f11(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 1 (N=3)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][10]
    if shift is None:
        shift = transforms.shifts[10][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][0]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.2, 0.4, 0.4])

    y = basic.zakharov(x_parts[0])
    y += basic.rosenbrock(x_parts[1])
    y += basic.rastrigin(x_parts[2])
    return y + 1100.0


def f12(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 2 (N=3)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][11]
    if shift is None:
        shift = transforms.shifts[11][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][1]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.3, 0.3, 0.4])

    y = basic.high_conditioned_elliptic(x_parts[0])
    y += basic.modified_schwefel(x_parts[1])
    y += basic.bent_cigar(x_parts[2])
    return y + 1200.0


def f13(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 3 (N=3)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][12]
    if shift is None:
        shift = transforms.shifts[12][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][2]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.3, 0.3, 0.4])

    y = basic.bent_cigar(x_parts[0])
    y += basic.rosenbrock(x_parts[1])
    y += basic.lunacek_bi_rastrigin(x_parts[2])
    return y + 1300.0


def f14(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 4 (N=4)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][13]
    if shift is None:
        shift = transforms.shifts[13][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][3]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.2, 0.2, 0.2, 0.4])

    y = basic.high_conditioned_elliptic(x_parts[0])
    y += basic.ackley(x_parts[1])
    y += basic.schaffers_f7(x_parts[2])
    y += basic.rastrigin(x_parts[3])
    return y + 1400.0


def f15(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 5 (N=4)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][14]
    if shift is None:
        shift = transforms.shifts[14][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][4]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.2, 0.2, 0.3, 0.3])

    y = basic.bent_cigar(x_parts[0])
    y += basic.h_g_bat(x_parts[1])
    y += basic.rastrigin(x_parts[2])
    y += basic.rosenbrock(x_parts[3])
    return y + 1500.0


def f16(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 6 (N=4)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][15]
    if shift is None:
        shift = transforms.shifts[15][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][5]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.2, 0.2, 0.3, 0.3])

    y = basic.expanded_schaffers_f6(x_parts[0])
    y += basic.h_g_bat(x_parts[1])
    y += basic.rosenbrock(x_parts[2])
    y += basic.modified_schwefel(x_parts[3])
    return y + 1600.0


def f17(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 7 (N=5)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][16]
    if shift is None:
        shift = transforms.shifts[16][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][6]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.1, 0.2, 0.2, 0.2, 0.3])

    y = basic.katsuura(x_parts[0])
    y += basic.ackley(x_parts[1])
    y += basic.expanded_griewanks_plus_rosenbrock(x_parts[2])
    y += basic.modified_schwefel(x_parts[3])
    y += basic.rastrigin(x_parts[4])
    return y + 1700.0


def f18(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 8 (N=5)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][17]
    if shift is None:
        shift = transforms.shifts[17][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][7]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.2, 0.2, 0.2, 0.2, 0.2])

    y = basic.high_conditioned_elliptic(x_parts[0])
    y += basic.ackley(x_parts[1])
    y += basic.rastrigin(x_parts[2])
    y += basic.h_g_bat(x_parts[3])
    y += basic.discus(x_parts[4])
    return y + 1800.0


def f19(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 9 (N=5)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][18]
    if shift is None:
        shift = transforms.shifts[18][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][8]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.2, 0.2, 0.2, 0.2, 0.2])

    y = basic.bent_cigar(x_parts[0])
    y += basic.rastrigin(x_parts[1])
    y += basic.expanded_griewanks_plus_rosenbrock(x_parts[2])
    y += basic.weierstrass(x_parts[3])
    y += basic.expanded_schaffers_f6(x_parts[4])
    return y + 1900.0


def f20(x, rotation=None, shift=None, shuffle=None):
    """
    Hybrid Function 10 (N=6)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotation (matrix): Optional rotation matrix. If None (default), the
            official matrix from the benchmark suite will be used.
        shift (array): Optional shift vector. If None (default), the official
            vector from the benchmark suite will be used.
        shuffle (array): Optionbal shuffle vector. If None (default), the
            official permutation vector from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotation is None:
        rotation = transforms.rotations[nx][19]
    if shift is None:
        shift = transforms.shifts[19][:nx]
    if shuffle is None:
        shuffle = transforms.shuffles[nx][9]

    x_transformed = transforms.shift_rotate(x, shift, rotation)
    x_parts = transforms.shuffle_and_partition(x_transformed, shuffle, [0.1, 0.1, 0.2, 0.2, 0.2, 0.2])

    y = basic.happy_cat(x_parts[0])
    y += basic.katsuura(x_parts[1])
    y += basic.ackley(x_parts[2])
    y += basic.rastrigin(x_parts[3])
    y += basic.modified_schwefel(x_parts[4])
    y += basic.schaffers_f7(x_parts[5])
    return y + 2000.0

def _calc_w(x, sigma):
    nx = x.shape[1]
    w = np.sum(x*x, axis=1)
    nzmask = w != 0
    w[nzmask] = ((1.0/w)**0.5)[nzmask] * np.exp(-w / (2.0*nx*sigma*sigma))[nzmask]
    w[~nzmask] = float('inf')
    return w


def _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases):
    nv = x.shape[0]
    nx = x.shape[1]

    N = len(funcs)
    vals = np.zeros((nv, N))
    w = np.zeros((nv, N))
    for i in range(0, N):
        x_shifted = x - np.expand_dims(shifts[i][:nx], 0)
        x_t = transforms.shift_rotate(x, shifts[i][:nx], rotations[i])
        vals[:, i] = funcs[i](x_t)
        w[:, i] = _calc_w(x_shifted, sigmas[i])
    w_sm = np.sum(w, axis=1)

    nz_mask = w_sm != 0.0
    w[nz_mask, :] /= w_sm[nz_mask, None]
    w[~nz_mask, :] = 1/N

    return np.sum(w * (lambdas*vals + biases), axis=1)


def _compose_hybrids(x, rotations, shifts, shuffles, funcs, sigmas, offsets, biases):
    nv = x.shape[0]
    nx = x.shape[1]

    N = len(funcs)
    vals = np.zeros((nv, N))
    w = np.zeros((nv, N))
    for i in range(0, N):
        x_shifted = x - np.expand_dims(shifts[i][:nx], 0)
        vals[:, i] = funcs[i](x, rotation=rotations[i], shift=shifts[i][:nx], shuffle=shuffles[i]) - offsets[i]
        w[:, i] = _calc_w(x_shifted, sigmas[i])
    w_sm = np.sum(w, axis=1)

    nz_mask = w_sm != 0.0
    w[nz_mask, :] /= w_sm[nz_mask, None]
    w[~nz_mask, :] = 1/N

    return np.sum(w * (vals + biases), axis=1)


def f21(x, rotations=None, shifts=None):
    """
    Composition Function 1 (N=3)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][0]
    if shifts is None:
        shifts = transforms.shifts_cf[0]

    funcs = [basic.rosenbrock, basic.high_conditioned_elliptic, basic.rastrigin]
    sigmas = np.array([10.0, 20.0, 30.0])
    lambdas = np.array([1.0, 1.0e-6, 1.0])
    biases = np.array([0.0, 100.0, 200.0])
    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2100


def f22(x, rotations=None, shifts=None):
    """
    Composition Function 2 (N=3)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][1]
    if shifts is None:
        shifts = transforms.shifts_cf[1]

    funcs = [basic.rastrigin, basic.griewank, basic.modified_schwefel]
    sigmas = np.array([10.0, 20.0, 30.0])
    lambdas = np.array([1.0, 10.0, 1.0])
    biases = np.array([0.0, 100.0, 200.0])

    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2200


def f23(x, rotations=None, shifts=None):
    """
    Composition Function 3 (N=4)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][2]
    if shifts is None:
        shifts = transforms.shifts_cf[2]

    funcs = [basic.rosenbrock, basic.ackley, basic.modified_schwefel, basic.rastrigin]
    sigmas = np.array([10.0, 20.0, 30.0, 40.0])
    lambdas = np.array([1.0, 10.0, 1.0, 1.0])
    biases = np.array([0.0, 100.0, 200.0, 300.0])
    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2300


def f24(x, rotations=None, shifts=None):
    """
    Composition Function 4 (N=4)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][3]
    if shifts is None:
        shifts = transforms.shifts_cf[3]

    funcs = [basic.ackley, basic.high_conditioned_elliptic, basic.griewank, basic.rastrigin]
    sigmas = np.array([10.0, 20.0, 30.0, 40.0])
    lambdas = np.array([1.0, 1.0e-6, 10.0, 1.0])
    biases = np.array([0.0, 100.0, 200.0, 300.0])
    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2400


def f25(x, rotations=None, shifts=None):
    """
    Composition Function 5 (N=5)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][4]
    if shifts is None:
        shifts = transforms.shifts_cf[4]

    funcs = [basic.rastrigin, basic.happy_cat, basic.ackley, basic.discus, basic.rosenbrock]
    sigmas = np.array([10.0, 20.0, 30.0, 40.0, 50.0])
    lambdas = np.array([10.0, 1.0, 10.0, 1.0e-6, 1.0])
    biases = np.array([0.0, 100.0, 200.0, 300.0, 400.0])
    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2500


def f26(x, rotations=None, shifts=None):
    """
    Composition Function 6 (N=5)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][5]
    if shifts is None:
        shifts = transforms.shifts_cf[5]

    funcs = [basic.expanded_schaffers_f6, basic.modified_schwefel, basic.griewank, basic.rosenbrock, basic.rastrigin]
    sigmas = np.array([10.0, 20.0, 20.0, 30.0, 40.0])
    # NOTE: the lambdas specified in the problem definitions (below) differ from
    # what is used in the code
    #lambdas = np.array([1.0e-26, 10.0, 1.0e-6, 10.0, 5.0e-4])
    lambdas = np.array([5.0e-4, 1.0, 10.0, 1.0, 10.0])
    biases = np.array([0.0, 100.0, 200.0, 300.0, 400.0])
    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2600


def f27(x, rotations=None, shifts=None):
    """
    Composition Function 7 (N=6)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][6]
    if shifts is None:
        shifts = transforms.shifts_cf[6]

    funcs = [
        basic.h_g_bat,
        basic.rastrigin,
        basic.modified_schwefel,
        basic.bent_cigar,
        basic.high_conditioned_elliptic,
        basic.expanded_schaffers_f6,
    ]
    sigmas = np.array([10.0, 20.0, 30.0, 40.0, 50.0, 60.0])
    lambdas = np.array([10.0, 10.0, 2.5, 1.0e-26, 1.0e-6, 5.0e-4])
    biases = np.array([0.0, 100.0, 200.0, 300.0, 400.0, 500.0])
    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2700


def f28(x, rotations=None, shifts=None):
    """
    Composition Function 8 (N=6)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][7]
    if shifts is None:
        shifts = transforms.shifts_cf[7]

    funcs = [
        basic.ackley,
        basic.griewank,
        basic.discus,
        basic.rosenbrock,
        basic.happy_cat,
        basic.expanded_schaffers_f6,
    ]
    sigmas = np.array([10.0, 20.0, 30.0, 40.0, 50.0, 60.0])
    lambdas = np.array([10.0, 10.0, 1.0e-6, 1.0, 1.0, 5.0e-4])
    biases = np.array([0.0, 100.0, 200.0, 300.0, 400.0, 500.0])
    return _composition(x, rotations, shifts, funcs, sigmas, lambdas, biases) + 2800


def f29(x, rotations=None, shifts=None, shuffles=None):
    """
    Composition Function 9 (N=3)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
        shuffles (array): Optional shuffle vectors (NxD). If None (default), the
            official permutation vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][8]
    if shifts is None:
        shifts = transforms.shifts_cf[8]
    if shuffles is None:
        shuffles = transforms.shuffles_cf[nx][0]

    funcs = [hybrid.f15, hybrid.f16, hybrid.f17]
    sigmas = np.array([10.0, 30.0, 50.0])
    biases = np.array([0.0, 100.0, 200.0])
    offsets = np.array([1500, 1600, 1700]) # subtract F* added at the end of the functions

    return _compose_hybrids(x, rotations, shifts, shuffles, funcs, sigmas, offsets, biases) + 2900


def f30(x, rotations=None, shifts=None, shuffles=None):
    """
    Composition Function 10 (N=3)

    Args:
        x (array): Input vector of dimension 2, 10, 20, 30, 50 or 100.
        rotations (matrix): Optional rotation matrices (NxDxD). If None
            (default), the official matrices from the benchmark suite will be
            used.
        shifts (array): Optional shift vectors (NxD). If None (default), the
            official vectors from the benchmark suite will be used.
        shuffles (array): Optional shuffle vectors (NxD). If None (default), the
            official permutation vectors from the benchmark suite will be used.
    """
    x = np.array(x)
    nx = x.shape[1]

    if rotations is None:
        rotations = transforms.rotations_cf[nx][9]
    if shifts is None:
        shifts = transforms.shifts_cf[9]
    if shuffles is None:
        shuffles = transforms.shuffles_cf[nx][1]

    funcs = [hybrid.f15, hybrid.f18, hybrid.f19]
    sigmas = np.array([10.0, 30.0, 50.0])
    biases = np.array([0.0, 100.0, 200.0])
    offsets = np.array([1500, 1800, 1900]) # subtract F* added at the end of the functions
    return _compose_hybrids(x, rotations, shifts, shuffles, funcs, sigmas, offsets, biases) + 3000




C:\Users\ivan\WORK\math\!NEW\cec2017\hybrid.py


In [7]:
all_functions = [
    f1,
    f2,
    f3,
    f4,
    f5,
    f6,
    f7,
    f8,
    f9,
    f10,
    f11,
    f12,
    f13,
    f14,
    f15,
    f16,
    f17,
    f18,
    f19,
    f20,
    f21,
    f22,
    f23,
    f24,
    f25,
    f26,
    f27,
    f28,
    f29,
    f30
]


In [5]:
all_functions = [
    #f2,
    f29,
    f30
]

In [6]:
import os
import numpy as np

# здесь должны быть импортированы/определены:
#  - VariableBounds
#  - GAConfig
#  - AdaptiveGeneticAlgorithm
#  - CEC-функции f1, f2, ..., f10 (или какие у тебя есть)
#  - список all_functions = [f1, f2, f3, ...]
#
# пример (зависит от твоей структуры проекта):
# from cec2017.basic import f1, f2, f3, f4, f5, f6, f7, f8, f9, f10
# all_functions = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10]


def make_fitness(func, dim: int):
    """
    Обёртка для CEC-функций: func(x) -> ndarray,
    берём первый элемент как скаляр.
    """
    def fitness(x_real: np.ndarray, x_disc: np.ndarray):
        x_real = np.asarray(x_real, dtype=float).reshape(1, dim)
        val = func(x_real)
        arr = np.asarray(val)
        return float(arr.ravel()[0])
    return fitness


def run_experiments():
    # список размерностей
    dims = [30]#, 50, 100]

    n_runs = 20
    n_generations = 200  # можешь увеличить, если нужно

    base_dir = "results_aga_dsp"
    os.makedirs(base_dir, exist_ok=True)

    for dim in dims:
        # --- настройка под текущую размерность ---
        print(f"\n=== DIMENSION {dim} ===\n")

        dim_dir = os.path.join(base_dir, f"D{dim}")
        os.makedirs(dim_dir, exist_ok=True)

        # границы для CEC (если другие - поменяй тут)
        bounds = VariableBounds(
            low=np.full(dim, -100.0),
            high=np.full(dim, 100.0)
        )

        # базовые настройки ГА (одинаковые для всех dim — при желании сделай условия)
        config = GAConfig(
            pop_size=2000,
            n_generations=n_generations,
            crossover_rate=0.8,
            mutation_rate=0.02,
            tournament_size=3,
            n_clusters=8,
            alpha_N=1.0,
            alpha_F=1.0,
            theta_ice=1.0,
            desired_step=0.01,
            min_pop_size=100,
            elite_archive_size=25,
        )

        # пробегаем по функциям из all_functions
        for fi, f in enumerate(all_functions, start=1):
            # берём имя из самой функции: f1, f2, f3, ...
            func_name = getattr(f, "__name__", f"f{fi}")

            func_dir = os.path.join(dim_dir, func_name)
            os.makedirs(func_dir, exist_ok=True)

            print(f"\n===== {func_name} (D={dim}) =====\n")

            for run_id in range(1, n_runs + 1):
                # здесь fi используется лишь как индекс в списке — это нормально,
                # seed просто должен быть уникален для разных функций и размерностей
                seed = 100000 * dim + 1000 * fi + run_id
                print(f"--- START {func_name} | D={dim} | run {run_id:02d} | seed {seed} ---")

                fitness_fn = make_fitness(f, dim)

                aga = AdaptiveGeneticAlgorithm(
                    fitness_fn=fitness_fn,
                    real_bounds=bounds,
                    disc_spec=None,  # дискретных переменных нет
                    config=config,
                    random_state=seed,
                )

                best_solution, best_fitness, history = aga.run()

                print(f"--- FINISHED {func_name} | D={dim} | run {run_id:02d} ---")
                print(f"best = {best_fitness:.6e}")

                # лог в файл: одна строка на поколение
                log_path = os.path.join(func_dir, f"run_{run_id:02d}.txt")
                with open(log_path, "w", encoding="utf-8") as fp:
                    fp.write(f"# {func_name}, D={dim}, run {run_id}\n")
                    fp.write(f"# best_fitness = {best_fitness:.12e}\n")
                    fp.write("# gen\tbest_fitness\tpop_before\tpop_after\tclusters\n")
                    for (gen, best_g, pop_b, pop_a, k) in history:
                        fp.write(f"{gen}\t{best_g:.12e}\t{pop_b}\t{pop_a}\t{k}\n")


if __name__ == "__main__":
    run_experiments()



=== DIMENSION 10 ===


===== f29 (D=10) =====

--- START f29 | D=10 | run 01 | seed 1001001 ---
Gen    1 | best = 3.276984e+03 | pop_before = 2000 | pop_after = 1543 | K = 6
Gen    2 | best = 3.412952e+03 | pop_before = 1543 | pop_after = 1016 | K = 5
Gen    3 | best = 3.526910e+03 | pop_before = 1016 | pop_after =  667 | K = 5
Gen    4 | best = 3.398101e+03 | pop_before =  667 | pop_after =  667 | K = 8
Gen    5 | best = 3.401108e+03 | pop_before =  667 | pop_after =  541 | K = 6
Gen    6 | best = 3.401738e+03 | pop_before =  541 | pop_after =  541 | K = 8
Gen    7 | best = 3.439437e+03 | pop_before =  541 | pop_after =  478 | K = 7
Gen    8 | best = 3.438510e+03 | pop_before =  478 | pop_after =  390 | K = 6
Gen    9 | best = 3.220185e+03 | pop_before =  390 | pop_after =  351 | K = 7
Gen   10 | best = 3.346491e+03 | pop_before =  351 | pop_after =  314 | K = 6
Gen   11 | best = 3.289093e+03 | pop_before =  314 | pop_after =  314 | K = 8
Gen   12 | best = 3.287150e+03 | pop_before =